In [1]:
# some imports
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

# Python ≥3.5 is required
import sys

assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn

assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf

assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
# % matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', size=12)
plt.rc('figure', figsize=(12, 5))

# Settings for the visualizations
#import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

import pandas as pd

pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

# Ignore useless warnings (see SciPy issue #5998)
import warnings

warnings.filterwarnings(action="ignore", message="^internal gelsd")
# Others
import cv2
import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from skimage.io import imread

 ## Import data

In [2]:
# Paths
X_train_path = './dataset/train/train/train/'
X_test_path = './dataset/test/test/test/'
dataset_csv = './dataset/train.txt'
dataset_train = './dataset/train/train'
dataset_test = './dataset/test/test'


In [3]:
_STOP = 5000


def preprocess(img_paths, dataset_path, preprocess_img_method=None):
    print("loading data")
    data = []
    stop = 0
    for img_path in tqdm.tqdm(img_paths):
        path = os.path.realpath(os.path.join(dataset_path, img_path[1:]))
        img = imread(path)
        if preprocess_img_method:
            img = preprocess_img_method(img)
        else:
            # img = cv2.resize(img,(224,224),cv2.INTER_AREA)
            img = img / 255.  #normalize
        data.append(img)
        stop += 1
        if (stop == _STOP):
            break
    print("loading data done")
    return data


def preprocess_img_vgg(img):
    return preprocess_input(img)


In [4]:
df = pd.read_csv(dataset_csv, delimiter='\ ', header=None)
y_train_full = np.array(df[1])[:_STOP]
img_paths = df[0]
X_train_full = preprocess(df[0], dataset_train, preprocess_img_method=None)
X_train_full = np.stack(X_train_full)

# Split dataset
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42,test_size=0.1)
X_train = X_train.astype("float32")
X_valid = X_valid.astype("float32")
y_train = y_train.astype("float32")
y_valid = y_valid.astype("float32")

C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


loading data


 33%|███▎      | 4999/15000 [00:03<00:07, 1361.07it/s]


loading data done


In [5]:
no_classes = np.max(y_train_full)
print(no_classes)

29


In [6]:
def root_mean_squared_error(y_true, y_pred):
     return tf.sqrt(tf.math.reduce_mean(tf.square(y_pred - y_true),axis=-1))

def metric_root_mean_squared_error(y_true,y_pred):
    y1 = tf.math.argmax(y_true,axis=-1)
    y2 = tf.math.argmax(y_pred,axis=-1)
    return tf.sqrt(tf.cast(tf.math.reduce_mean(tf.square(y1-y2)),tf.float64))

In [7]:
def get_uncompiled_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=[158, 158]))
    model.add(tf.keras.layers.Dense(512, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dense(512, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(no_classes + 1, activation="softmax"))
    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer=keras.optimizers.SGD(lr=1e-5, decay=(5 * 1e-5), momentum=0.95),  # Optimizer
        # Loss function to minimize
        loss=keras.losses.SparseCategoricalCrossentropy(),
        # List of metrics to monitor
        metrics=["accuracy"]
    )
    return model

In [8]:
model = get_compiled_model()

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 24964)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12782080  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0

In [10]:
# Prepare a directory to store all the checkpoints.
checkpoint_dir = "./ckpt"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()

### Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
# configure early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
###

### init callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        # The saved model name will include the current epoch.
        filepath=checkpoint_dir+ "/ckpt_{epoch}_loss={loss:.2f}",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        mode="min",
        verbose=1,
    ),
    early_stopping
]

In [11]:
# model = make_or_restore_model()
print("Fit model on training data")
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=50,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_valid, y_valid)
    # validation_split=0.2
    # callbacks=callbacks
)

Fit model on training data
Epoch 1/50
141/141 [==============================] - 1s 6ms/step - loss: 3.4590 - accuracy: 0.0344 - val_loss: 3.3986 - val_accuracy: 0.0360

KeyboardInterrupt: 

In [ ]:
predict = model.predict(X_train[:1])
predict1 = model.predict(X_train[2][np.newaxis, :, :])
print(predict)

In [ ]:
preds = []
n= len(y_valid)
for data in X_valid[:n]:
    pred = model.predict(data[np.newaxis, :, :])
    preds.append((np.argmax(pred)))

##
vals = y_valid[:n]
rmse = [np.sqrt(np.sum(np.square(preds-vals))/n)]
print(rmse)

In [ ]:
print(rmse)